In [1]:
import os, sys, glob
import csv
import cv2
import numpy as np
import glob

np.set_printoptions(suppress=True)


In [11]:
data_dir =  '/home/staff1/ctorney/data/horses/departures/'
data_dir =  '/home/ctorney/euclid/data/horses/departures/'
escaped=False
width = 1920
height = 1080
fps = 30
gap = 240
display = True

filelist = glob.glob(data_dir + "*.mp4")
# track list format -> time, track_id, xmin, xmax, ymin, ymax, horse_id
for input_file in filelist:
    direct, ext = os.path.split(input_file)
    noext, _ = os.path.splitext(ext)
    data_file = data_dir + '/tracks/' +  noext + '_POS.txt'
    out_data_file = data_dir + '/tracks/' +  noext + '_POS_M.txt'
    
    if os.path.isfile(out_data_file):
        continue
    video_file = data_dir + '/tracks/' +  noext + '_TR.avi'
    
    cap = cv2.VideoCapture(video_file)
    fCount = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    tracks = np.genfromtxt(data_file,delimiter=",",dtype='float')
    z = np.zeros((len(tracks),1), dtype='float')
    z[:,0] = tracks[:,1]
    tracks= np.append(tracks, z, axis=1)
    trackids = np.unique(tracks[:,1]).tolist()
    alltrackids = np.unique(tracks[:,1])
    for t in alltrackids:
        thisTrack = tracks[tracks[:,1]==t]
        if len(thisTrack)<10:
            tracks = tracks[tracks[:,1]!=t]
            trackids.remove(t)
        elif ((thisTrack[-1,0]-thisTrack[0,0])<5*fps):
            tracks = tracks[tracks[:,1]!=t]
            trackids.remove(t)
    newid=0
    start_times = np.zeros(len(trackids))
    stop_times = np.zeros(len(trackids))
    for t in range(len(trackids)):
        start_times[t] = np.min(tracks[tracks[:,1]==trackids[t],0])
        stop_times[t] = np.max(tracks[tracks[:,1]==trackids[t],0])
    
    current_track=0
    for t in range(len(trackids)):
        current_track = tracks[tracks[:,1]==trackids[t],6][0]
        diffs = np.abs(stop_times[t]-start_times)
        for t2 in range(len(trackids)):
            if t==t2:
                continue
            if diffs[t2]<gap:
                
                frName = 'is ' + str(trackids[t2]) + ' the same as ' + str(trackids[t]) + ' ? y or n'
                cv2.destroyAllWindows()
                cv2.namedWindow(frName, flags =  cv2.WINDOW_NORMAL)
                cv2.resizeWindow(frName, 1920,1080)
                startFrame = max(0,stop_times[t]-150)
                stopFrame = min(fCount-1, stop_times[t]+150)
                cap.set(cv2.CAP_PROP_POS_FRAMES,startFrame)
                
                
                yes=False
                while True:
                    thisFrame = cap.get(cv2.CAP_PROP_POS_FRAMES)
                    if thisFrame>stopFrame:
                        cap.set(cv2.CAP_PROP_POS_FRAMES,startFrame)
                    _, frame = cap.read()
                    cv2.imshow(frName,frame)
                    k = cv2.waitKey(10)

                    if k==ord('y'):
                        yes=True
                        break
                    if k==ord('n'):
                        break
                    if k==27:    # Esc key to stop
                        escaped=True
                        break 
                    
                if yes:
                    tracks[tracks[:,1]==trackids[t2],6]=current_track
                    break
                if escaped:
                    break
        
        if escaped:
            break
    alltrackids = np.unique(tracks[:,6])
    current_track=0
    for t in alltrackids:
        tracks[tracks[:,6]==t]=current_track
        current_track+=1
        
    if escaped:
        cv2.destroyAllWindows()
        break
    
    
    with open(out_data_file, "w") as output:
        writer = csv.writer(output, lineterminator='\n')
        writer.writerows(tracks)


In [7]:
for input_file in filelist:
    print(input_file)
    direct, ext = os.path.split(input_file)
    noext, _ = os.path.splitext(ext)
    data_file = data_dir + '/tracks/' +  noext + '_POS.txt'
    out_data_file = data_dir + '/tracks/' +  noext + '_POS_M.txt'
    
    #if os.path.isfile(out_data_file):
    break

/home/ctorney/euclid/data/horses/departures/180708-4m2.mp4


In [9]:
os.path.isfile(out_data_file)

True

In [49]:
tracks = np.genfromtxt(data_file,delimiter=",",dtype='float')
z = np.zeros((len(tracks),1), dtype='float')
z[:,0] = tracks[:,1]
tracks= np.append(tracks, z, axis=1)

In [73]:
tracks = np.genfromtxt(data_file,delimiter=",",dtype='float')

In [78]:
video_file

'/home/ctorney/euclid/data/horses/departures//tracks/180708-4m2_TR.avi'

In [ ]:
thisTrack[-1,0]-thisTrack[0,0]

In [26]:
len(thisTrack)

2

In [29]:
trackids.remove(20)